**Capstone Code: Section 1, Fetch City Geo Data and Restaurant List from Foursquare**

*Import libraries*

In [1]:
import numpy as np                                                       # library to handle data in a vectorized manner
from numpy.random import randint 

import pandas as pd                                                      # library for data analsysis
import json                                                              # library to handle JSON files
from pandas.io.json import json_normalize                                # normalize JSON file data
import requests                                                          # library to handle fetching api requests

from geopy.geocoders import Nominatim                                    # convert an address into latitude and longitude values
geolocator = Nominatim(user_agent="foursquare_agent")

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"                          # display all requested output in pandas format

*Set-up Foursquare credentials and required variables*

In [2]:
CLIENT_ID = 'MJ1EJQRTUUE3ZGU4NE2DV2P2HZWO0U5E3QU1EFLOT51GQQC3'           # your Foursquare ID
CLIENT_SECRET = 'UFY2KX53GD0PRWMZQ0T4BU2ANOHX24RZM2HGXNHAWODLRGL1'       # your Foursquare Secret
print('Your credentials are saved.')

VERSION = '20201015'                                                     # required API version
LIMIT = 200                                                              # limit API results per endpoint
# variables
search_query = 'Mexican'                                                 # search parameter
pluralNames = "Mexican%20Restaurants"                                    # search parameter
subs = "Mexican Restaurant"                                              # set search variable
lcat = "categories"                                                      #   
llat = "location.lat"
llng = "location.lng"
ladr = "location.address"
capMex = pd.DataFrame(columns=['id','name','address','city'])            # dataFrame reserved for function
print('Your fixed variables are saved.\n')

Your credentials are saved.
Your fixed variables are saved.



**Load Michigan city lat, lng, pop and growth rate data into a dataframe**

In [3]:
cities = pd.read_csv(r'C:\Users\Jim_2\Downloads\simplemaps\uscities.csv') # location of https://simplemaps.com/data/us-cities download
cities = cities.reset_index(drop=True)                                    # add index back onto data
cities = cities.loc[(cities['city'].isin(['Detroit','Kalamazoo',\
    'Lansing','Grand Rapids','Flint','Ann Arbor','Midland',\
    'Jackson','Battle Creek','Saginaw'])) & (cities['state_id'] =="MI")]  # locate data for Michigan cities

cities2 = cities[['city','lat','lng','population']].copy()                # select required columns from dataframe
cities2['growth_rate'] = [.005,.0917,-.047,.029,.065,.059,-.048,-.011,-.015,-.006]  # add googled growth rate to dataframe

print("\nMichigan Cities and Geo Data:\n")

for c, row in cities2.iterrows():                                         # loop through cities2 to gather foursquare data
    lat = row['lat']                                                      # set latitude variable for foursquare function
    lng = row['lng']                                                      # set longitude variable for foursquare function
    city = row['city']                                                    # set city variable for foursquare function
    
    if city == "Detroit":
        radius = 40000                                                    # set radius value for Detroit
    elif city == "Grand Rapids":
        radius = 25000                                                    # set radius value for Grand Rapids
    else:
        radius = 15000                                                    # set radius for all other cities
        
   # it returns a dataframe that provides a list of Mexican restaurants in specific metro areas, their id, categories, address, lat and lng
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}&pluralNames={}'.format(CLIENT_ID, CLIENT_SECRET, lat, lng, VERSION, search_query, radius, LIMIT, pluralNames)
    
    global mexOnly,capMex                                                 # set global variables inside function
           
    results = requests.get(url).json()                                    # make the GET request
    results2 = results['response']['venues']                              # grab data
    results3 = pd.json_normalize(results2)                                # tranform venues into a dataframe
    results3['city'] = city
    
    mexOnly = results3[['id','name',lcat,ladr,'city']].copy()             # select required Foursquare data
    mexOnly.columns = ['id','name','categories','address','city']         # rename columns
    
    mex2 = mexOnly[lcat].copy()                                           # get categories object column from mex2
    mex2 = pd.DataFrame(mex2).astype(str)                                 # change to string datatype
    mex2["value"]= mex2['categories'].str.find(subs)                      # add bool value column with identify "Mexican Restaurant"
    
    mexOnly['value'] = mex2['value']                                      # add value column to mexOnly
       
    mexOnly = mexOnly[mexOnly.value != -1]                                # remove row values that are equal to -1 (not Mex restaurants)
    mexOnly = mexOnly[['id','name','address','city']]                     # provides list of only Mexican restaurants in city
    mexOnly = mexOnly.reset_index(drop=True)                              # resets row index
    
    
    # saving output data while looping through Michigan cities
    city2 = city[:2]
    onlyCity = 'only%s'%city2
    cityurl = 'C:\\Users\Jim_2\Desktop\python_projects\cities\{}.csv'.format(onlyCity)
    mexOnly.to_csv(cityurl, index=False, header=True)                    # saving file to disk
    capMex = capMex.append(mexOnly)                                      # saving current city data to existing data on capMex
    mexCount = len(results3)
    cities2.loc[c,'count'] = round(mexCount)

cities2.to_csv(r'C:\Users\Jim_2\Desktop\python_projects\cities2.csv', index=False, header=True) # saving updated cities2 to disk
capMex.to_csv(r'C:\Users\Jim_2\Desktop\python_projects\capMex.csv', index=False, header=True) # saving capMex to disk

capMex.head()
capMex.shape
cities2


Michigan Cities and Geo Data:



,id,name,address,city
0,4a46bd39f964a5205fa91fe3,Mexican Town Restaurant,3457 Bagley St,Detroit
1,4ad36269f964a52012e420e3,Mexican Village Restaurant,2600 Bagley St,Detroit
2,4b50da97f964a520663527e3,Qdoba Mexican Grill,5779 E 12 Mile Rd,Detroit
3,4b104dcaf964a520656d23e3,On The Border Mexican Grill & Cantina,3310 Fairlane Dr,Detroit
4,4b49321df964a520ad6926e3,Mexican Gardens,15950 Eureka Rd,Detroit


(152, 4)

,city,lat,lng,population,growth_rate,count
13,Detroit,42.3834,-83.1024,3506126.0,0.0050,50.0
73,Grand Rapids,42.9620,-85.6562,609314.0,0.0917,42.0
130,Flint,43.0235,-83.6922,332190.0,-0.0470,16.0
135,Lansing,42.7142,-84.5601,324286.0,0.0290,30.0
136,Ann Arbor,42.2755,-83.7312,322267.0,0.0650,22.0
209,Kalamazoo,42.2749,-85.5882,215193.0,0.0590,16.0
377,Saginaw,43.4199,-83.9501,117958.0,-0.0480,6.0
532,Jackson,42.2431,-84.4037,87110.0,-0.0110,1.0
620,Battle Creek,42.2986,-85.2296,76514.0,-0.0150,8.0
848,Midland,43.6241,-84.2319,58787.0,-0.0060,5.0
